In [200]:
from dataclasses import dataclass, field
from datetime import datetime
import time
from functools import total_ordering
import os
from sortedcontainers import SortedList
from enum import Enum, auto
import random

In [2]:
class OrderIdGenerator:
    def __init__(self):
        self.filepath = os.path.join(os.getcwd(), "current_state.txt")
        self.id_counter = self.load_state()
        self.iterations_since_last_save = 0
        self.save_frequency = 100

    def __iter__(self):
        return self

    def __next__(self):
        current_id = self.id_counter
        self.id_counter += 1
        self.iterations_since_last_save += 1
        if self.iterations_since_last_save >= self.save_frequency:
            self.save_state()
            self.iterations_since_last_save = 0

        return current_id

    def save_state(self):
        with open(self.filepath, "w") as f:
            f.write(str(self.id_counter))

    def load_state(self):
        if not os.path.exists(self.filepath):
            return 0
        with open(self.filepath, "r") as f:
            saved_state = f.read()
            return int(saved_state)

    def reset_state(self):
        self.id_counter = 0
        self.save_state()


g_obj = OrderIdGenerator()
next(g_obj)

0

In [3]:


class OrderStatus(Enum):
    CREATED = auto()
    PARTIALLY_FILLED = auto()
    FILLED = auto()
    CANCELLED = auto()
    

In [120]:
@dataclass
class Order:
    order_type: str = field(compare=False)
    security_name: str = field(compare=False)
    price: float = field(compare=True)
    volume: int = field(compare=False)
    owner_id: int = field(compare=False)
    status: int = field(compare=False, default=OrderStatus.CREATED)
    id: int = field(compare=False, default_factory=g_obj.__next__)
    created: float = field(compare=False, default_factory=time.time)
    updated: float = field(compare=False, default_factory=time.time)
    modified: float = field(compare=False, default_factory=time.time)

    def __eq__(self, other):
        if isinstance(other, (int, float)):
            return self.price == other
        if isinstance(other, Order):
            return self.price == other.price
        return NotImplemented

    def __lt__(self, other):
        if isinstance(other, (int, float)):
            return self.price < other
        if isinstance(other, Order):
            return self.price < other.price
        return NotImplemented

    def __gt__(self, other):
        if isinstance(other, (int, float)):
            return self.price > other
        if isinstance(other, Order):
            return self.price > other.price
        return NotImplemented

    def __le__(self, other):
        if isinstance(other, (int, float)):
            return self.price <= other
        if isinstance(other, Order):
            return self.price <= other.price
        return NotImplemented

    def __ge__(self, other):
        if isinstance(other, (int, float)):
            return self.price >= other
        if isinstance(other, Order):
            return self.price >= other.price
        return NotImplemented

    def __repr__(self):
        return (
            f"Order № {self.id} \n"
            f"Order Type {self.order_type} \n"
            f"Security: {self.security_name} \n"
            f"Price: {self.price} \n"
            f"Volume: {self.volume} \n"
            f"Owner: {self.owner_id} \n"
            f"Status: {self.status} \n"
            f"Created: {self.created} \n"
            f"Modified: {self.modified} \n"
            f"Last update: {self.updated} \n"
        )

    def __post_init__(self):
        if self.volume <= 0:
            raise ValueError("Volume must be positive")
        if self.price <= 0:
            raise ValueError("Price must be positive")
        if self.order_type not in ["ask", "bid"]:
            raise ValueError("Order type must be 'ask' or 'bid'")
        if self.status not in OrderStatus:
            raise ValueError("Invalid order status")

    def __setattr__(self, key, value):
        super().__setattr__(key, value)
        if key == "price":
            super().__setattr__("modified", time.time())
        if key != "updated":  # Избегаем рекурсивного вызова
            super().__setattr__("updated", time.time())

    def json(self):
        return {
            "order_type": self.order_type,
            "security_name": self.security_name,
            "price": self.price,
            "volume": self.volume,
            "owner_id": self.owner_id,
            "status": self.status,
            "id": self.id,
            "created": self.created,
            "modified": self.modified,
            "updated": self.updated,
        }

What functions I want to keep in Orderbook and what to outside the Book?

We can create api wrapper that will keep all processes 

In [ ]:
import logging

def logging_decorator(func):
    def wrapper(*args, **kwargs):
        # Logging setup
        logger = logging.getLogger(func.__name__)
        logger.setLevel(logging.INFO)
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        stream_handler = logging.StreamHandler()
        stream_handler.setFormatter(formatter)
        logger.addHandler(stream_handler)
        
        # Logging before function execution
        logger.info(f"Calling function {func.__name__} with args: {args}, kwargs: {kwargs}")
        
        # Function execution
        result = func(*args, **kwargs)
        
        # Logging after function execution
        logger.info(f"Function {func.__name__} executed successfully. Result: {result}")
        
        return result
    
    return wrapper


In [46]:
class OperationType(Enum):
    add = auto()
    fill = auto()
    cancel = auto()

<OperationType.add: 1>

In [42]:
def my_decorator(func):
    def wrapper(self, *args, **kwargs):
        print(func.__name__)
        print(self.x)
        result = func(self, *args, **kwargs)
        # Действия после вызова функции
        return result

    return wrapper


class MyClass:
    def __init__(self):
        self.x = 5
    
    
    @my_decorator
    def my_method(self):
        return self.x

c = MyClass()
c.my_method()

my_method
5


5

In [169]:
class OrderList:
    def __init__(self, order_list=None):
        self.__order_list = SortedList()
        self.__ids = {}
        if order_list:
            for order in order_list:
                self.add(order)

    def add(self, order):
        self.__order_list.add(order)
        self.__ids[order.id] = order

    def bisect_left(self, order, right = True):
        cid = self.__order_list.bisect_left(order)
        if right:
            return self.__order_list[cid:]
        else:
            return self.__order_list[:cid]

    def bisect_right(self, order, left = True):
        cid = self.__order_list.bisect_right(order)
        if left:
            return self.__order_list[:cid]
        else:
            return self.__order_list[cid:]

    def pop(self, index=-1):
        order = self.__order_list.pop(index)
        del self.__ids[order.id]
        return order

    def pop_by_id(self, order_id):
        order = self.__ids[order_id]
        self.remove(order)
        return order

    def remove(self, order):
        self.__order_list.remove(order)
        del self.__ids[order.id]

    def clear(self):
        self.__order_list.clear()
        self.__ids.clear()

    # def __getitem__(self, index):
    #     return self.__order_list[index]
    
    
    def __getitem__(self, order_id):
        return self.__ids[order_id]

    def __iter__(self):
        return iter(self.__order_list)
    
    def __len__(self):
        return len(self.__order_list)

    def __repr__(self):
        return str(self.__order_list)

In [170]:
a = Order(order_type="ask", security_name="AAPL", price=100, volume=10, owner_id=1)
ol = OrderList([a])

a.id

172

In [172]:
b = Order(order_type="ask", security_name="AAPL", price=103, volume=10, owner_id=1)
time.sleep(1)
c = Order(order_type="ask", security_name="AAPL", price=99, volume=10, owner_id=1)
ol.add(b)
ol.add(c)


In [176]:
ol[173]

Order № 173 
Order Type ask 
Security: AAPL 
Price: 103 
Volume: 10 
Owner: 1 
Status: OrderStatus.CREATED 
Created: 1706795657.116274 
Modified: 1706795657.116274 
Last update: 1706795657.116274 

In [ ]:
ol.price = 200 # Не работает, т.к. SortedList не поддерживает изменение элементов
ol[0]

In [213]:
import pprint


class OrderBook:
    def __init__(self):

        self.ask = OrderList()

        self.bid = OrderList()
        self.tape = []


    def add(self, order):
        matched = []
        if order.order_type == "ask":
            matched = self.bid.bisect_left(order)
        else:
            matched = self.ask.bisect_right(order)
        matched.sort(key=lambda x: x.modified)
        while order.volume > 0 and matched:
            current = matched.pop(0)
            deal_price = current.price
            self.fill(order, current, deal_price)
        if order.volume > 0:
            if order.order_type == "ask":
                self.ask.add(order)
            elif order.order_type == "bid":
                self.bid.add(order)
            else:
                raise ValueError("Order type must be 'ask' or 'bid'")


    def fill(self, order, matched_order, price):
        volume = 0
        if matched_order.volume >= order.volume:
            matched_order.volume -= order.volume
            volume = order.volume
            order.volume = 0
            order.status = OrderStatus.FILLED
            if matched_order.volume == 0:
                self.remove_order(matched_order)
            else:
                self.get_order(matched_order).volume = matched_order.volume
        else:
            order.volume -= matched_order.volume
            self.remove_order(matched_order)
            volume = matched_order.volume
        self.tape.append(
            {
                "order": order.id,
                "contr_order": matched_order.id,
                "price": price,
                "volume": volume,
                "time": time.time(),
            }
        )

    def find_by_id(self, order_id):
        if order_id in self.ask:
            return self.ask[order_id]
        elif order_id in self.bid:
            return self.bid[order_id]
        else:
            raise ValueError("Order with such id doesn't exist")

    def get_order(self, order):
        if order.order_type == "ask":
            return self.ask[order.id]
        elif order.order_type == "bid":
            return self.bid[order.id]
        else:
            raise ValueError("Order type must be 'ask' or 'bid'")

    def remove_order(self, order):
        if order.order_type == "ask":
            self.ask.remove(order)
        elif order.order_type == "bid":
            self.bid.remove(order)
        else:
            raise ValueError("Order type must be 'ask' or 'bid'")

    def cancel(self, order):
        self.get_order(order).status = OrderStatus.CANCELLED
        self.remove_order(order)

    def modify(self, order, price):
        self.get_order(order).price = price

    def proceede(self):
        order_tape = self.order_tape.copy()
        self.order_tape.clear()
        return order_tape


od = OrderBook()

for _ in range(20):
    tmp = Order(
        order_type="bid",
        security_name="AAPL",
        price=random.randint(50, 105),
        volume=random.randint(1, 100),
        owner_id=random.randint(6, 11),
    )

    od.add(tmp)
for _ in range(20):
    tmp = Order(
        order_type="ask",
        security_name="AAPL",
        price=random.randint(90, 120),
        volume=random.randint(1, 100),
        owner_id=random.randint(1, 5),
    )

    od.add(tmp)

    # ...

print("Order Tape:")
pprint.pprint(od.tape)

print("Ask Orders:")
pprint.pprint([f"{order.id} - {order.price}" for order in od.ask])

print("Bid Orders:")
pprint.pprint([f"{order.id} - {order.price}" for order in od.bid])

# print(od.tape)
# print([f"{order.id} - {order.price}" for order in od.ask])
# print([f"{order.id} - {order.price}" for order in od.bid])

Order Tape:
[{'contr_order': 535,
  'order': 542,
  'price': 102,
  'time': 1706796799.5627055,
  'volume': 93},
 {'contr_order': 521,
  'order': 548,
  'price': 100,
  'time': 1706796799.5637028,
  'volume': 12},
 {'contr_order': 532,
  'order': 548,
  'price': 97,
  'time': 1706796799.5637028,
  'volume': 37},
 {'contr_order': 531,
  'order': 548,
  'price': 99,
  'time': 1706796799.5637028,
  'volume': 35},
 {'contr_order': 531,
  'order': 556,
  'price': 99,
  'time': 1706796799.5637028,
  'volume': 64},
 {'contr_order': 523,
  'order': 557,
  'price': 91,
  'time': 1706796799.5637028,
  'volume': 1},
 {'contr_order': 537,
  'order': 557,
  'price': 97,
  'time': 1706796799.5637028,
  'volume': 55}]
Ask Orders:
['557 - 90',
 '560 - 94',
 '556 - 99',
 '559 - 99',
 '554 - 102',
 '545 - 103',
 '546 - 104',
 '551 - 105',
 '543 - 107',
 '549 - 107',
 '558 - 107',
 '555 - 110',
 '550 - 111',
 '544 - 113',
 '547 - 115',
 '552 - 117',
 '553 - 117',
 '541 - 118']
Bid Orders:
['534 - 50',
 '

In [208]:
print([f"id {order.id} volume {order.volume}"for order in od.ask])
print([f"id {order.id} volume {order.volume}"for order in od.bid])

['id 266 volume 9', 'id 268 volume 9', 'id 270 volume 15', 'id 264 volume 2', 'id 269 volume 17', 'id 267 volume 24', 'id 265 volume 7', 'id 263 volume 9']
['id 280 volume 50', 'id 273 volume 22', 'id 274 volume 24', 'id 275 volume 35', 'id 278 volume 46', 'id 279 volume 31', 'id 277 volume 28', 'id 276 volume 21', 'id 272 volume 5']


In [209]:
print([f"{order.id} - {order.price}" for order in od.ask])
print([f"{order.id} - {order.price}" for order in od.bid])

['266 - 89', '268 - 92', '270 - 98', '264 - 101', '269 - 106', '267 - 113', '265 - 116', '263 - 122']
['280 - 69', '273 - 70', '274 - 70', '275 - 70', '278 - 71', '279 - 72', '277 - 76', '276 - 79', '272 - 87']


In [187]:
for _ in range(10):
    tmp = Order(order_type="ask", security_name="AAPL", price=random.randint(75, 125), volume=random.randint(1, 25), owner_id=random.randint(1, 5))
    
    od.add(tmp)

[(76, 5), (81, 14), (82, 13), (89, 18), (93, 10), (98, 24), (101, 11), (110, 3), (118, 15), (120, 16)]


ValueError: 212 not in list

In [199]:
od.tape

[]

AttributeError: 'int' object has no attribute 'order_type'

In [ ]:
import unittest

class OrderBookTests(unittest.TestCase):
    def setUp(self):
        self.order_book = OrderBook()

    def test_add_order(self):
        order = Order(order_type="ask", security_name="AAPL", price=100, volume=10, owner_id=1)
        self.order_book.add(order)
        self.assertEqual(len(self.order_book.ask), 1)

    def test_fill_order(self):
        order1 = Order(order_type="ask", security_name="AAPL", price=100, volume=10, owner_id=1)
        order2 = Order(order_type="bid", security_name="AAPL", price=100, volume=5, owner_id=2)
        self.order_book.add(order1)
        self.order_book.add(order2)
        self.assertEqual(len(self.order_book.ask), 0)
        self.assertEqual(len(self.order_book.bid), 0)
        self.assertEqual(len(self.order_book.tape), 1)

    def test_cancel_order(self):
        order = Order(order_type="ask", security_name="AAPL", price=100, volume=10, owner_id=1)
        self.order_book.add(order)
        self.order_book.cancel(order.id)
        self.assertEqual(order.status, OrderStatus.CANCELLED)
        self.assertEqual(len(self.order_book.ask), 0)

    def test_modify_order(self):
        order = Order(order_type="ask", security_name="AAPL", price=100, volume=10, owner_id=1)
        self.order_book.add(order)
        self.order_book.modify(order.id, 200)
        self.assertEqual(order.price, 200)

    def tearDown(self):
        self.order_book = None

unittest.main()


Order lifecycle

Creation -> Added to Orderbook -> Matching -> Repeat till filled -> Every n-ish minutes are 

In [54]:
sl = SortedList()
a = Order(order_type="ask", security_name="AAPL", price=100, volume=10, owner_id=1)
time.sleep(1)
b = Order(order_type="ask", security_name="AAPL", price=101, volume=50, owner_id=1)
time.sleep(1)
c = Order(order_type="ask", security_name="AAPL", price=102, volume=80, owner_id=1)

In [51]:
omap = {}
sl = SortedList()
omap[a.id] = a
sl.add(omap[a.id])
are_same = (omap[a.id] is sl[0])
print(are_same)


True


In [78]:
tl = OrderList([a, b])
tl.add(c)
tl

SortedList([Order № 13 
Order Type ask 
Security: AAPL 
Price: 100 
Volume: 10 
Owner: 1 
Status: OrderStatus.CREATED 
Created: 1706783465.5706851 
Last update: 1706783465.5706851 
, Order № 14 
Order Type ask 
Security: AAPL 
Price: 101 
Volume: 50 
Owner: 1 
Status: OrderStatus.CREATED 
Created: 1706783466.5719347 
Last update: 1706783466.5719347 
, Order № 15 
Order Type ask 
Security: AAPL 
Price: 102 
Volume: 80 
Owner: 1 
Status: OrderStatus.CREATED 
Created: 1706783467.5725536 
Last update: 1706783467.5725536 
])

In [83]:
tl

SortedList([Order № 13 
Order Type ask 
Security: AAPL 
Price: 100 
Volume: 10 
Owner: 1 
Status: OrderStatus.CREATED 
Created: 1706783465.5706851 
Last update: 1706783465.5706851 
, Order № 14 
Order Type ask 
Security: AAPL 
Price: 101 
Volume: 50 
Owner: 1 
Status: OrderStatus.CREATED 
Created: 1706783466.5719347 
Last update: 1706783466.5719347 
])

In [49]:
print(a)
time.sleep(1)
a.status = OrderStatus.FILLED
print(a)

Order № 7 
Order Type ask 
Security: AAPL 
Price: 100 
Volume: 10 
Owner: 1 
Status: OrderStatus.CREATED 
Created: 1706739080.7954504 
Last update: 1706739080.7954504 

Order № 7 
Order Type ask 
Security: AAPL 
Price: 100 
Volume: 10 
Owner: 1 
Status: OrderStatus.FILLED 
Created: 1706739080.7954504 
Last update: 1706739108.0433347 



In [14]:
sl.clear()
sl.add(a)
sl.add(b)
sl.add(c)
sl.bisect_left(Order(order_type="ask", security_name="AAPL", price=101, volume=50, owner_id=1))

1

In [16]:
sl.remove(b)

In [17]:
sl

SortedList([Order № 1 
Order Type ask 
Security: AAPL 
Price: 100 
Volume: 10 
Owner: 1 
Status: OrderStatus.CREATED 
Created: 1706699471.7772381 
Last update: 1706699471.7772381 
, Order № 3 
Order Type ask 
Security: AAPL 
Price: 102 
Volume: 80 
Owner: 1 
Status: OrderStatus.CREATED 
Created: 1706699473.7799664 
Last update: 1706699473.7799664 
])

In [15]:
kk = sl[:2]
kk.sort(key=lambda x: x.last_update)
print(kk)

[Order № 1 
Order Type ask 
Security: AAPL 
Price: 100 
Volume: 10 
Owner: 1 
Status: OrderStatus.CREATED 
Created: 1706699471.7772381 
Last update: 1706699471.7772381 
, Order № 2 
Order Type ask 
Security: AAPL 
Price: 101 
Volume: 50 
Owner: 1 
Status: OrderStatus.CREATED 
Created: 1706699472.7790332 
Last update: 1706699472.7790332 
]


In [29]:
from bintrees import FastRBTree, RBTree

In [30]:
tree = RBTree()
tree.insert(1, a)
tree.insert(2, b)
tree.insert(3, c)

tree

RBTree({1: Order № 7 
Order Type ask 
Security: AAPL 
Price: 100 
Volume: 10 
Owner: 1 
Status: OrderStatus.CREATED 
Created: 1706372748.6780784 
Last update: 1706372748.6780784 
, 2: Order № 8 
Order Type ask 
Security: AAPL 
Price: 101 
Volume: 50 
Owner: 1 
Status: OrderStatus.CREATED 
Created: 1706372749.68011 
Last update: 1706372749.68011 
, 3: Order № 9 
Order Type ask 
Security: AAPL 
Price: 102 
Volume: 80 
Owner: 1 
Status: OrderStatus.CREATED 
Created: 1706372750.6813135 
Last update: 1706372750.6813135 
})

In [31]:
tree.

AttributeError: 'RBTree' object has no attribute 'bisect_left'

In [14]:
sl.add(a)
sl.add(b)
sl.add(c)
sl

TypeError: '<' not supported between instances of 'Order' and 'Order'

In [ ]:
from enum import Enum

class MarketSecurities(Enum):
    APPLE = 1
    GOOGLE = 2
    MICROSOFT = 3
    AMAZON = 4

# Example usage
security_id = MarketSecurities.APPLE.value
print(security_id)  # Output: 1


In [ ]:
from enum import Enum, auto

id_dict = {"AAPLE": 1, "GOOGLE": 2, "MICROSOFT": 3, "AMAZON": 4}

class MarketSecurities(Enum):
    pass

for name, value in id_dict.items():
    setattr(MarketSecurities, name, value)

# Example usage
security_id = MarketSecurities.APPLE.value
print(security_id)  # Output: 1


In [19]:
a = Order("bid", "Microsoft", 100, 10, 1)
print(a.fill(15))
a.status
# b = Order("bid", "Microsoft", 110, 10, 1)
# c = Order("bid", "Microsoft", 95, 10, 1)
# print(a)
# print(b)
# print(c)

30


<OrderStatus.PARTIALLY_FILLED: 2>

<OrderStatus.CANCELLED: 4>

In [11]:
from bintrees import AVLTree, RBTree

In [15]:
from sortedcontainers import SortedList

In [40]:
sl = SortedList([a, b, c])

In [42]:
d= Order("bid", "Microsoft", 80, 100, 1)

In [43]:
print(sl)
sl.add(d)
sl

SortedList([Order № 16 
Order Type bid 
Security: Microsoft 
Price: 95 
Volume: 10 
Owner: 1 
Status: 0 
Time: 1706132732.8304594 
, Order № 14 
Order Type bid 
Security: Microsoft 
Price: 100 
Volume: 10 
Owner: 1 
Status: 0 
Time: 1706132732.8304594 
, Order № 15 
Order Type bid 
Security: Microsoft 
Price: 110 
Volume: 10 
Owner: 1 
Status: 0 
Time: 1706132732.8304594 
])


SortedList([Order № 17 
Order Type bid 
Security: Microsoft 
Price: 80 
Volume: 100 
Owner: 1 
Status: 0 
Time: 1706133128.3177133 
, Order № 16 
Order Type bid 
Security: Microsoft 
Price: 95 
Volume: 10 
Owner: 1 
Status: 0 
Time: 1706132732.8304594 
, Order № 14 
Order Type bid 
Security: Microsoft 
Price: 100 
Volume: 10 
Owner: 1 
Status: 0 
Time: 1706132732.8304594 
, Order № 15 
Order Type bid 
Security: Microsoft 
Price: 110 
Volume: 10 
Owner: 1 
Status: 0 
Time: 1706132732.8304594 
])

In [ ]:
class OrderTree(SortedList):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._max_id = 0

    def insert(self, order):
        super().insert(order.id, order)
        self._max_id = max(self._max_id, order.id)

    def remove(self, order):
        super().remove(order.id)
        if order.id == self._max_id:
            self._max_id = max(self.keys())

    def get_max_id(self):
        return self._max_id